In [ ]:
from pymongo import MongoClient
import os
from pprint import pprint
import json
from dotenv import load_dotenv
from bson.objectid import ObjectId
from bson import json_util
import re

In [ ]:
# Load environment variables
load_dotenv()

In [ ]:
mongo_username = "root"
mongo_password = os.environ.get("PRODMONGOPASSWORD")
host = "localhost"
port = "27019"  # dev

mongoclient_dev = MongoClient(
    "mongodb://%s:%s@%s:%s" % (mongo_username, mongo_password, host, port)
)
# Send a ping to the server to see if it's responding
try:
    mongoclient_dev.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


## Get Galveston Mapping Set

In [ ]:
# Select your database
db = mongoclient_dev['dfr3db']
# Select your collection
collection = db['MappingSet']

# Perform the query to find one document with the specified ObjectId
mapping_doc = collection.find_one({"_id": ObjectId("6303e51bd76c6d0e1f6be080")})

fragilities = [ i['entry']['Non-Retrofit Fragility ID Code'] for i in mapping_doc['mappings']]

fragilities

In [ ]:
def get_fragility_mapping(fragility_id):
    collection = db['FragilitySet']
    fragility_doc = collection.find_one({"_id": ObjectId(fragility_id)})
    return fragility_doc

# Save the fragility mapping to a json file
for id in fragilities:
    fragility_doc = get_fragility_mapping(id)
    os.makedirs("fragilities_backup", exist_ok=True)  # Create the directory if it doesn't exist
    with open(f"fragilities_backup/fragility_{id}.json", "w") as f:
        # Convert the document to a JSON string and write it to a file
        f.write(json_util.dumps(fragility_doc, indent=4))
        print(f"Saved fragility_{id}.json")

In [ ]:
pprint(get_fragility_mapping(fragilities[0]))

In [ ]:
def regex_replace_in_json(data, pattern, replacement):
    if isinstance(data, dict):
        for key, value in data.items():
            data[key] = regex_replace_in_json(value, pattern, replacement)
    elif isinstance(data, list):
        data = [regex_replace_in_json(item, pattern, replacement) for item in data]
    elif isinstance(data, str):
        data = re.sub(pattern, replacement, data)
    return data
    
# Replace the string in the json file
pattern = r"math\.floor\(ffe_elev\)"
replacement = r"round(ffe_elev)"
collection = db['FragilitySet']

for id in fragilities:
    frag_json = get_fragility_mapping(id)
    frag_json_updated = regex_replace_in_json(frag_json, pattern, replacement)
    # Update the document in the collection
    collection.update_one({"_id": ObjectId(id)}, {"$set": frag_json_updated})



In [ ]:
# Test the replacement
frag_json = collection.find_one({"_id": ObjectId(fragilities[0])})
pprint(frag_json)

In [ ]:
# Replace document in the collection with backup
for id in fragilities:
    with open(f"fragilities_backup/fragility_{id}.json", "r") as f:
        frag_json = json_util.loads(f.read())
        collection.update_one({"_id": ObjectId(id)}, {"$set": frag_json})
        print(f"Updated fragility_{id}.json")